# Intro to Natural Language Processing
Welcome to NLP. NLP aims to enable computers to understand, interpret, and generate human language in a way that is both meaningful and useful.
Applications of NLP range from sentiment analysis, machine translation, chatbots, speech recognition, text summarization, and information retrieval.
In this notebook, we'll dive into the world of text analysis.
We will explore ways to extract meaning from text, and build a model that can differentiate between positive and negative sentiment in movie reviews.
We'll be using a simplistic technique called Bag of Words,
 which involves representing text as numerical vectors of words represented their frequency and index.




# Data
To start off let's get some data. The data we are going to use is from the IMDb Dataset.
The IMDB dataset is a large dataset of movie reviews from the website IMDb.
It contains 50,000 movie reviews, half of which are labeled as positive and half as negative,
and is often used as a benchmark dataset for natural language processing tasks.
Grab the dataset from [kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) if you haven't already.

In [7]:
import pandas as pd

# load dataset into a pandas dataframe
df = pd.read_csv('IMDB Dataset.csv')

In [2]:
#  Let's see what's in it!
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing Data
In order to get usable data, we must transform the data to be suitable for analysis.
We'll be using some regular expression to clean out unwanted strings and
the `CountVectorizer` from scikit-learn to transform the collection of text
 into a matrix of token counts where each row represents a document and
  each column represents a unique word in the document collection.
  Let's see it in action.

In [8]:
import re
from pprint import pprint

# before
pprint(df.iloc[0]['review'])

# remove non characters from review
regex = re.compile('<\\w+ /?>|[^\\w \']|\\d|_')
df['review'].replace(regex, ' ', regex=True, inplace=True)

# after
pprint(df.iloc[0]['review'])

('One of the other reviewers has mentioned that after watching just 1 Oz '
 "episode you'll be hooked. They are right, as this is exactly what happened "
 'with me.<br /><br />The first thing that struck me about Oz was its '
 'brutality and unflinching scenes of violence, which set in right from the '
 'word GO. Trust me, this is not a show for the faint hearted or timid. This '
 'show pulls no punches with regards to drugs, sex or violence. Its is '
 'hardcore, in the classic use of the word.<br /><br />It is called OZ as that '
 'is the nickname given to the Oswald Maximum Security State Penitentary. It '
 'focuses mainly on Emerald City, an experimental section of the prison where '
 'all the cells have glass fronts and face inwards, so privacy is not high on '
 'the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, '
 'Christians, Italians, Irish and more....so scuffles, death stares, dodgy '
 'dealings and shady agreements are never far away.<br /><br />I would

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# stop_words='english' removes common English words like "a" or "the' from the text
vectorizer = CountVectorizer(stop_words='english', lowercase=True)
vectorizer.fit(df['review'])

CountVectorizer(stop_words='english')

In [12]:
# see some of the tokens it collected
vectorizer.get_feature_names_out()

array(['aa', 'aaa', 'aaaaaaaaaaaahhhhhhhhhhhhhh', ..., 'þór', 'יגאל',
       'כרמון'], dtype=object)

# Dataset
Now let's make it into a pytorch `Dataset`.

In [9]:
import torch
from torch.utils.data import Dataset
from pandas.core.series import Series

class IMDBDataset(Dataset):
    regex = re.compile('<\\w+ /?>|[^\\w \']|\\d|_')

    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.vectorizer = CountVectorizer(stop_words='english', lowercase=True, max_df=.5, min_df=10)

        # clean data
        self.df['review'].replace(IMDBDataset.regex, ' ', regex=True, inplace=True)

        # fit vectorizer
        self.vectorizer.fit(self.df['review'])

    def __getitem__(self, index: int):
        review: str

        reviews = self.df.iloc[index]['review']

        # this just allows us to keep slicing in our dataset
        reviews = reviews if isinstance(reviews, Series) else [reviews]

        # transform text into tensors
        vector = self.vectorizer.transform(reviews).toarray()
        X = torch.squeeze(torch.tensor(vector)).type(torch.FloatTensor)

        # assign values to our classes
        values = {
            'negative': 0,
            'positive': 1
        }

        sentiments = self.df.iloc[index]['sentiment']

        # again, this just allows us to keep slicing in our dataset
        Y = list(map(lambda sentiment: values[sentiment], sentiments)) if isinstance(sentiments, Series) else [values[sentiments]]

        # convert to tensors
        Y = torch.tensor(Y).type(torch.FloatTensor)

        return X, Y

    def __len__(self):
        return len(self.df)

    @property
    def classes(self):
        return 'negative', 'positive'

In [10]:
dataset = IMDBDataset(df)

In [24]:
dataset[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor([1.]))

# Model
Next, let's make a super simple logistic regression model. The model should receive a tensor of term frequencies and output a value between 0 and 1.

In [16]:
from torch import nn
class LogisticRegression(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, X: torch.Tensor):
        return torch.sigmoid(self.linear(X))

In [107]:
model = LogisticRegression(len(dataset.vectorizer.get_feature_names_out()), 1)

# Hyper parameters
Set some hyper parameters

In [21]:
epochs = 3
batch_size = 16
lr = 1e-3
num_folds = 3

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cross Validation
Before we get to the training loop, let's understand cross-validation.
Cross-validation is a technique used to evaluate a machine learning
model's performance by splitting the dataset into multiple subsets or folds.
The model is trained on a portion of the data and tested on the remaining fold, which is repeated for each fold.
The overall performance is then calculated by averaging the performance of each fold.

In [30]:
from sklearn.model_selection import KFold

# k-fold cross validation
kf = KFold(n_splits=num_folds)

# Metrics
We'll need a way to score the performance of our model.
We'll benchmark our model with `AUROC` from torchmetrics.
The `AUROC` score summarizes the Receiver Operating Characteristic Curve
 into a single number that describes the performance of a model for multiple thresholds at the same time.
 Notably, an `AUROC` score of 1 is a perfect score and an `AUROC` score of 0.5 corresponds to random guessing.

In [31]:
from torchmetrics import AUROC

metric = AUROC(task='binary').to(device)

# Training
Our training loop performs k-fold cross validation, with each fold iterating through training and evaluating a logistic regression model on a binary classification task, with the `Adam` optimizer and `BCELoss` function, and computing the average train and test loss.

In [121]:
from typing import List
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import Adam
from copy import deepcopy

def cross_validate(model: nn.Module, dataset: Dataset, *, num_folds=3, epochs=3, lr=1e-3, batch_size=16):

    # original state dict
    state_dict = deepcopy(model.state_dict())

    # keep track of model scores
    scores: List[float] = []

    kf = KFold(n_splits=num_folds)
    metric = AUROC(task='binary').to(device)

    for fold, (train_indices, test_indices) in enumerate(kf.split(dataset)):

        # make samplers that samples elements randomly from a given list of indices without replacement.
        train_sampler = SubsetRandomSampler(train_indices)
        test_sampler = SubsetRandomSampler(test_indices)

        # data loaders
        train_loader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            sampler=train_sampler,
        )
        test_loader = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            sampler=test_sampler,
        )

        # instantiate model
        model.load_state_dict(deepcopy(state_dict))

        # loss function
        criterion = nn.BCELoss()

        # optimizer
        optimizer = Adam(model.parameters(), lr=lr)

        # training loop
        for epoch in range(epochs):
            total_train_loss = 0
            total_test_loss = 0

            # training
            print('training ', end='')
            model.train()
            for i, (X, Y) in enumerate(train_loader):
                # print progress
                if i % (len(train_loader) // 10) == 0:
                    print('.', end='')

                # forward pass
                outputs: torch.Tensor = model(X.to(device))

                # calculate loss
                loss: torch.Tensor = criterion(outputs, Y.to(device))

                # zero out accumulated gradients
                optimizer.zero_grad()

                # backpropagation
                loss.backward()
                total_train_loss += loss.item()

                # update weights and biases
                optimizer.step()

            # testing
            print('\ntesting  ', end='')
            model.eval()
            total_score = 0
            for i, (X, Y) in enumerate(test_loader):
                # print progress
                if i % (len(test_loader) // 10) == 0:
                    print('.', end='')

                outputs: torch.Tensor = model(X.to(device))
                loss: torch.Tensor = criterion(outputs, Y.to(device))

                # update loss and scores
                total_test_loss += loss.item()
                total_score += metric(outputs, Y.to(device))
            scores.append(total_score / len(test_loader))
            print(f'\nFold {fold} | Epoch {epoch} | train loss {total_train_loss / len(train_loader):.2f} | '
                  f'test loss {total_test_loss / len(test_loader):.2f} | '
                  f'auroc score {total_score / len(test_loader):.2f}')
        print(f'\n{"-" * 90}\n')
    return scores

In [ ]:
model.to(device)
scores = cross_validate(
    model=model,
    dataset=dataset,
    num_folds=num_folds,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr
)

# Confidence Interval
Calculate range of values that is likely to contain
the true population parameter with a certain level of
confidence based on a sample of data.

In [102]:
import numpy as np
from scipy import stats

# confidence interval function
def confidence_interval(data: List[float]):
    sem = stats.sem(data)
    if sem == 0:
        return data[0], data[0]
    return stats.t.interval(confidence=.95, df=len(data)-1, loc=np.mean(data), scale=sem)

In [103]:
confidence_interval(scores)

(96, 96)

# Improvement
Now let's try to improve our model. Let's add more non-linearity with ReLU activation function!

In [115]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.relu = nn.ReLU()

    def forward(self, X: torch.Tensor):
        return torch.sigmoid(self.relu(self.linear(X)))

In [ ]:
model = LogisticRegression(len(dataset.vectorizer.get_feature_names_out()), 1).to(device)
scores = cross_validate(
    model=model,
    dataset=dataset,
    num_folds=num_folds,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr
)
confidence_interval(scores)

Did it improve?

Try increasing the model complexity with more layers.

In [117]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features: int, out_features: int, hidden_units: int = 32):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, out_features),
            nn.ReLU(),
        )

    def forward(self, X: torch.Tensor):
        return torch.sigmoid(self.block(X))

In [122]:
model = LogisticRegression(len(dataset.vectorizer.get_feature_names_out()), 1).to(device)
scores = cross_validate(
    model=model,
    dataset=dataset,
    num_folds=num_folds,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr
)
confidence_interval(scores)

training ...........
testing  ...........
Fold 0 | Epoch 0 | train loss 0.52 | test loss 0.50 | auroc score 0.90
training ...........
testing  ...........
Fold 0 | Epoch 1 | train loss 0.44 | test loss 0.54 | auroc score 0.90
training ...........
testing  ...........
Fold 0 | Epoch 2 | train loss 0.41 | test loss 0.59 | auroc score 0.89

------------------------------------------------------------------------------------------

training ...........
testing  ...........
Fold 1 | Epoch 0 | train loss 0.52 | test loss 0.50 | auroc score 0.90
training ...........
testing  ...........
Fold 1 | Epoch 1 | train loss 0.44 | test loss 0.54 | auroc score 0.90
training ...........
testing  ...........
Fold 1 | Epoch 2 | train loss 0.42 | test loss 0.64 | auroc score 0.91

------------------------------------------------------------------------------------------

training ...........
testing  ...........
Fold 2 | Epoch 0 | train loss 0.52 | test loss 0.50 | auroc score 0.90
training ...........
te

C:\Users\iwlpo\Documents\Coding\PycharmProjects\upic_data_science\nlp_hug\venv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


..........
Fold 2 | Epoch 1 | train loss 0.44 | test loss 0.51 | auroc score 0.89
training ...........
testing  ...........
Fold 2 | Epoch 2 | train loss 0.42 | test loss 0.56 | auroc score 0.90

------------------------------------------------------------------------------------------



(0.8927225795144409, 0.9044627222662598)

How was it this time?

Try one more time with regularization (prevent overfitting and improve generalization performance) using `Dropout`.
`Dropout` will randomly turn off some nodes during training.

In [123]:
class LogisticRegression(nn.Module):
    def __init__(self, in_features: int, out_features: int, hidden_units: int = 32):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(in_features, hidden_units),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_units, out_features),
            nn.ReLU(),
            nn.Dropout(),
        )

    def forward(self, X: torch.Tensor):
        return torch.sigmoid(self.block(X))

In [ ]:
model = LogisticRegression(len(dataset.vectorizer.get_feature_names_out()), 1).to(device)
scores = cross_validate(
    model=model,
    dataset=dataset,
    num_folds=num_folds,
    epochs=epochs,
    batch_size=batch_size,
    lr=lr
)
confidence_interval(scores)

# Conclusion
 As you can see the rest of the models didn't do that well. Machine learning requires lots of experimenting.
It often requires trying out different models, hyperparameters, and preprocessing techniques to achieve optimal results.
This is only the start of NLP. Throughout the workshop you may find other approaches to this problem.